## Automated Timetabling

Imports for the required libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import xml.etree.ElementTree as ET

### Preparing the course information to create the required files

Firstly, scrape the module information from the Copmuter Science Programme

In [2]:
# URL of the Computer Science Undergraduate Course in UCD
url = 'https://hub.ucd.ie/usis/!W_HU_MENU.P_PUBLISH?p_tag=MAJR&MAJR=CSSA' 

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

table = soup.find('table', id='CB555-7Q')

if table is not None:
    rows = table.find_all('tr')
    with open('moduleUrls.txt', 'w') as file:
        counter = 1
        for row in rows:
            module_types = row.find('th')
            if module_types is not None and module_types.get_text(strip=True) != "":
                counter += 1
                if counter % 2 == 1:
                    if counter != 3:
                        file.write("\n")
                    types = module_types.get_text(strip=True)
                    file.write(types)
                    file.write("\n")

            first_td = row.find('td')


            if first_td is not None and first_td.get_text(strip=True) != "":
                module_code = first_td.get_text(strip=True)

                module_url = f"https://hub.ucd.ie/usis/!W_HU_MENU.P_PUBLISH?p_tag=MODULE&MODULE={module_code}&TERMCODE=202300\n"

                file.write(module_url)

    print("Module URLs written to moduleUrls.txt")


Module URLs written to moduleUrls.txt


Now, scrape the required information about each module from the urls 

In [3]:
with open('ModuleUrls.txt', 'r') as module_urls_file:
    module_urls = module_urls_file.readlines()

with open('ScrapedInformation.txt', 'w') as file:
    file.write("")

for url in module_urls:
    url = url.strip()

    if(url.strip().startswith("https")): 
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        headers = soup.find_all('h1')

        if len(headers) >= 2:
            second_header = headers[1] 

            with open('ScrapedInformation.txt', 'a', encoding='utf-8') as file:
                file.write(second_header.text.strip() + "\n")

        dl_element = soup.find('dl')

        if dl_element is not None:
            lines = dl_element.text.strip().split('\n')

            with open('ScrapedInformation.txt', 'a', encoding='utf-8') as file:
                file.write(lines[0] + ' ' + lines[1] + '\n')
                file.write(lines[6] + ' ' + lines[7] + '\n')
                file.write(lines[8] + ' ' + lines[9] + '\n')
                file.write(lines[10] + ' ' + lines[11] + '\n')
                file.write(lines[12] + ' ' + lines[13] + '\n')
                file.write('\n') 

            table = soup.find('table', id='CB100-20Q')

            if table is not None:
                rows = table.find_all('tr')

                with open('ScrapedInformation.txt', 'a', encoding='utf-8') as file:
                    for i in range(1, min(7, len(rows))):
                        file.write(rows[i].text.strip().replace('\n', ' ') + '\n')
                    file.write("\n")
            else:
                print(f"Table with ID 'CB100-20Q' not found in URL: {url}")

            table = soup.find('table', id='CB100-98Q')

            if table is not None:
                rows = table.find_all('tr')

                with open('ScrapedInformation.txt', 'a', encoding='utf-8') as file:
                    for i in range(1, min(7, len(rows))):
                        file.write(rows[i].text.strip().replace('\n', ' ') + '\n')
                    
                    file.write("\n ******************************* \n")
                    file.write("\n")
            else:
                print(f"Table with ID 'CB100-98Q' not found in URL: {url}")
        else:
            print(f"dl element not found in URL: {url}")
    else:
        with open('ScrapedInformation.txt', 'a', encoding='utf-8') as file:
            file.write(url + '\n')

print("All information written to ScrapedInformation.txt")

All information written to ScrapedInformation.txt


We will now run code to split the modules and their relevant information into two new files, for Autumn modules and Spring modules

In [4]:
def parse_module_info(module_info):
    """Parse the module information and return a dictionary of attributes."""
    attributes = {}
    lines = module_info.split('\n')
    for line in lines:
        if line.strip():
            key_value = line.split(':')
            if len(key_value) == 2:
                key = key_value[0].strip()
                value = key_value[1].strip()
                attributes[key] = value
    return attributes

with open('SpringModules.txt', 'w') as file:
    file.write("")
with open('AutumnModules.txt', 'w') as file:
    file.write("")

input_file = 'ScrapedInformation.txt'
spring_output_file = 'SpringModules.txt'
autumn_output_file = 'AutumnModules.txt'

with open(input_file, 'r') as f:
    data = f.read()

modules = data.split('\n\n ******************************* \n\n')

for module in modules:
    lines = module.split('\n')
    for line in lines:
        if 'Stage ' in line:
            with open(spring_output_file, 'a') as f:
                f.write(line + '\n')
            with open(autumn_output_file, 'a') as f:
                f.write(line + '\n')

    if 'Trimester:' in module:
        attributes = parse_module_info(module)
        if 'Trimester' in attributes:
            trimester = attributes['Trimester']
            output_file = spring_output_file if trimester == 'Spring' else autumn_output_file

            with open(output_file, 'a') as f:
                f.write(module.strip() + '\n\n')
                f.write(' ******************************* \n\n')

print("Modules split into files for both Autumn and Spring")


Modules split into files for both Autumn and Spring


### Creating the xml files to be inputted to Unitime

Firstly, we must create the Session Setup files. We will do so for Autumn and Spring.
Note: Trimester dates must be manually changed from the springOriginal.xml file and autumnOriginal.xml files. These must be changed under the < session> and < datePatterns> tags.

In [5]:
pattern = re.compile(r'^[A-Z]{4}\d{5}$')
altpattern = re.compile(r'^[A-Z]{3}\d{5}$')

with open('BasicAutumnSession.xml', 'r') as old_autumn_file, open('AutumnSession.xml', 'w') as new_autumn_file:
    seen_modules = set()    
    for line in old_autumn_file:
        new_autumn_file.write(line)

        if '<subjectAreas>' in line:
            with open('AutumnModules.txt', 'r') as input_autumn_file:
                for input_line in input_autumn_file:
                    input_line = input_line.strip()

                    if input_line and pattern.match(input_line.split(' ', 1)[0]):  
                        parts = input_line.split(' ', 1)
                        module_code = parts[0]
                        module_title = parts[1]
                            
                        if module_code not in seen_modules:
                            seen_modules.add(module_code)
                            new_line = f'    <subjectArea abbreviation="{module_code}" title="{module_title}" department="0001"/>\n'
                            new_autumn_file.write(new_line)

                    elif input_line and altpattern.match(input_line.split(' ', 1)[0]):
                        parts = input_line.split(' ', 1)
                        module_code = parts[0]
                        module_title = parts[1]
                            
                        
                        if module_code not in seen_modules:
                            seen_modules.add(module_code)
                            new_line = f'    <subjectArea abbreviation="{module_code}" title="{module_title}" department="0001"/>\n'
                            new_autumn_file.write(new_line)
print("Autumn file Created")


with open('BasicSpringSession.xml', 'r') as old_spring_file, open('SpringSession.xml', 'w') as new_spring_file:
    seen_modules = set()
    for line in old_spring_file:
        new_spring_file.write(line)

        if '<subjectAreas>' in line:
            with open('SpringModules.txt', 'r') as input_spring_file:
                for input_line in input_spring_file:
                    input_line = input_line.strip()
                    if input_line and pattern.match(input_line.split(' ', 1)[0]):  
                        parts = input_line.split(' ', 1)
                        module_code = parts[0]
                        module_title = parts[1]
                            
                        if module_code not in seen_modules:
                            seen_modules.add(module_code)
                            new_line = f'    <subjectArea abbreviation="{module_code}" title="{module_title}" department="0001"/>\n'
                            new_spring_file.write(new_line)                    
                    elif input_line and altpattern.match(input_line.split(' ', 1)[0]): 
                        parts = input_line.split(' ', 1)
                        module_code = parts[0]
                        module_title = parts[1]
                            
                        if module_code not in seen_modules:
                            seen_modules.add(module_code)
                            new_line = f'    <subjectArea abbreviation="{module_code}" title="{module_title}" department="0001"/>\n'
                            new_spring_file.write(new_line)   
print("Spring File Created")


Autumn file Created
Spring File Created


We need to create a staffImport.xml to add the staff members to the system


We can fistly do this for Autumn

In [6]:
pattern = re.compile(r'^Module Coordinator:\s*(.*)$')
externalId = 1000

xml_content = '<?xml version="1.0" encoding="UTF-8"?>\n'
xml_content += '<!DOCTYPE offerings PUBLIC "-//UniTime//DTD University Course Timetabling/EN" "http://www.unitime.org/interface/CourseOfferingExport.dtd">\n\n'
xml_content += '<!-- \n'
xml_content += ' * Licensed to The Apereo Foundation under one or more contributor license\n'
xml_content += ' * agreements. See the NOTICE file distributed with this work for\n'
xml_content += ' * additional information regarding copyright ownership.\n'
xml_content += ' *\n'
xml_content += ' * The Apereo Foundation licenses this file to you under the Apache License,\n'
xml_content += ' * Version 2.0 (the "License"); you may not use this file except in\n'
xml_content += ' * compliance with the License. You may obtain a copy of the License at:\n'
xml_content += ' *\n'
xml_content += ' * http://www.apache.org/licenses/LICENSE-2.0\n'
xml_content += ' *\n'
xml_content += ' * Unless required by applicable law or agreed to in writing, software\n'
xml_content += ' * distributed under the License is distributed on an "AS IS" BASIS,\n'
xml_content += ' * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n'
xml_content += ' *\n'
xml_content += ' * See the License for the specific language governing permissions and\n'
xml_content += ' * limitations under the License.\n'
xml_content += ' *\n'
xml_content += ' -->\n\n'
xml_content += '<staff campus="UCD" year="2025" term="Autumn">'

processed_coordinators = set()

with open('AutumnModules.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

current_module_info = []
for line in lines:
    line = line.strip()
    if line == '*******************************':
        if current_module_info:
            for subpart in current_module_info:
                match = pattern.match(subpart)

                if match:
                    coordinator_info = match.group(1).strip()
                    parts = coordinator_info.split()

                    if parts:
                        if parts[0] == 'Assoc' and parts[1] == 'Professor':
                            acadTitle = 'Mr'
                            positionType = 'ASSOC_PROF'
                            if len(parts) == 5:
                                firstName = parts[2]
                            else:
                                firstName = parts[-2]
                            lastName = parts[-1]
                        elif parts[0] == 'Dr':
                            acadTitle = 'Dr'
                            positionType = 'PROF'
                            if len(parts) == 4:
                                firstName = parts[1]
                            else:
                                firstName = parts[-2]
                            lastName = parts[-1]
                        elif parts[0] == 'Mr':
                            acadTitle = 'Mr'
                            positionType = 'PROF'
                            if len(parts) == 4:
                                firstName = parts[1]
                            else:
                                firstName = parts[-2]
                            lastName = parts[-1]
                        elif parts[0] == 'Professor':
                            acadTitle = 'Mr'
                            positionType = 'PROF'
                            if len(parts) == 4:
                                firstName = parts[1]
                            else:
                                firstName = parts[-2]
                            lastName = parts[-1]

                    if coordinator_info not in processed_coordinators:
                        xml_content += f'\n\t<staffMember externalId="{externalId}" firstName="{firstName}" lastName="{lastName}" acadTitle="{acadTitle}" positionType="{positionType}" department="0001"/>'
                        processed_coordinators.add(coordinator_info)
                        coordinator_info = None

                    externalId += 1

            current_module_info = []
        continue
    else:
        current_module_info.append(line)

xml_content += '\n</staff>\n'

with open('AutumnStaffImport.xml', 'w', encoding='utf-8') as output_file:
    output_file.write(xml_content)

print('XML file "AutumnStaffImport.xml" has been generated successfully.')


XML file "AutumnStaffImport.xml" has been generated successfully.


Now, for Spring

In [7]:
pattern = re.compile(r'^Module Coordinator:\s*(.*)$')
externalId = 1000

xml_content = '<?xml version="1.0" encoding="UTF-8"?>\n'
xml_content += '<!DOCTYPE offerings PUBLIC "-//UniTime//DTD University Course Timetabling/EN" "http://www.unitime.org/interface/CourseOfferingExport.dtd">\n\n'
xml_content += '<!-- \n'
xml_content += ' * Licensed to The Apereo Foundation under one or more contributor license\n'
xml_content += ' * agreements. See the NOTICE file distributed with this work for\n'
xml_content += ' * additional information regarding copyright ownership.\n'
xml_content += ' *\n'
xml_content += ' * The Apereo Foundation licenses this file to you under the Apache License,\n'
xml_content += ' * Version 2.0 (the "License"); you may not use this file except in\n'
xml_content += ' * compliance with the License. You may obtain a copy of the License at:\n'
xml_content += ' *\n'
xml_content += ' * http://www.apache.org/licenses/LICENSE-2.0\n'
xml_content += ' *\n'
xml_content += ' * Unless required by applicable law or agreed to in writing, software\n'
xml_content += ' * distributed under the License is distributed on an "AS IS" BASIS,\n'
xml_content += ' * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n'
xml_content += ' *\n'
xml_content += ' * See the License for the specific language governing permissions and\n'
xml_content += ' * limitations under the License.\n'
xml_content += ' *\n'
xml_content += ' -->\n\n'
xml_content += '<staff campus="UCD" year="2025" term="Spring">'

processed_coordinators = set()

with open('SpringModules.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

current_module_info = []
for line in lines:
    line = line.strip()
    if line == '*******************************':
        if current_module_info:
            for subpart in current_module_info:
                match = pattern.match(subpart)

                if match:
                    coordinator_info = match.group(1).strip()
                    parts = coordinator_info.split()

                    if parts:
                        if parts[0] == 'Assoc' and parts[1] == 'Professor':
                            acadTitle = 'Mr'
                            positionType = 'ASSOC_PROF'
                            if len(parts) == 5:
                                firstName = parts[2]
                            else:
                                firstName = parts[-2]
                            lastName = parts[-1]
                        elif parts[0] == 'Dr':
                            acadTitle = 'Dr'
                            positionType = 'PROF'
                            if len(parts) == 4:
                                firstName = parts[1]
                            else:
                                firstName = parts[-2]
                            lastName = parts[-1]
                        elif parts[0] == 'Mr':
                            acadTitle = 'Mr'
                            positionType = 'PROF'
                            if len(parts) == 4:
                                firstName = parts[1]
                            else:
                                firstName = parts[-2]
                            lastName = parts[-1]
                        elif parts[0] == 'Professor':
                            acadTitle = 'Mr'
                            positionType = 'PROF'
                            if len(parts) == 4:
                                firstName = parts[1]
                            else:
                                firstName = parts[-2]
                            lastName = parts[-1]

                    if coordinator_info not in processed_coordinators:
                        xml_content += f'\n\t<staffMember externalId="{externalId}" firstName="{firstName}" lastName="{lastName}" acadTitle="{acadTitle}" positionType="{positionType}" department="0001"/>'
                        processed_coordinators.add(coordinator_info)
                        coordinator_info = None
                    
                    externalId += 1

            current_module_info = []
        continue
    else:
        current_module_info.append(line)

xml_content += '\n</staff>\n'

with open('SpringStaffImport.xml', 'w', encoding='utf-8') as output_file:
    output_file.write(xml_content)

print('XML file "SpringStaffImport.xml" has been generated successfully.')


XML file "SpringStaffImport.xml" has been generated successfully.


We now want to create the Instructional Offerings

First, for Autumn

In [8]:
pattern = re.compile(r'^([A-Z]{4}\d+)\s(.+)$')
altpattern = re.compile(r'^([A-Z]{3}\d+)\s(.+)$')
credit_pattern = re.compile(r'^Credits:\s+(\d+\.\d+)$')
lecture_pattern = re.compile(r'^Lectures\s+(\d+)$')
lecturer = re.compile(r'^Module Coordinator:\s*(.*)$')
seen_modules = set()
offering_counter = 1
class_counter = 1000

xml_content = '<?xml version="1.0" encoding="UTF-8"?>\n'
xml_content += '<!DOCTYPE offerings PUBLIC "-//UniTime//DTD University Course Timetabling/EN" "http://www.unitime.org/interface/CourseOfferingExport.dtd">\n\n'
xml_content += '<!-- \n'
xml_content += ' * Licensed to The Apereo Foundation under one or more contributor license\n'
xml_content += ' * agreements. See the NOTICE file distributed with this work for\n'
xml_content += ' * additional information regarding copyright ownership.\n'
xml_content += ' *\n'
xml_content += ' * The Apereo Foundation licenses this file to you under the Apache License,\n'
xml_content += ' * Version 2.0 (the "License"); you may not use this file except in\n'
xml_content += ' * compliance with the License. You may obtain a copy of the License at:\n'
xml_content += ' *\n'
xml_content += ' * http://www.apache.org/licenses/LICENSE-2.0\n'
xml_content += ' *\n'
xml_content += ' * Unless required by applicable law or agreed to in writing, software\n'
xml_content += ' * distributed under the License is distributed on an "AS IS" BASIS,\n'
xml_content += ' * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n'
xml_content += ' *\n'
xml_content += ' * See the License for the specific language governing permissions and\n'
xml_content += ' * limitations under the License.\n'
xml_content += ' *\n'
xml_content += ' -->\n\n'
xml_content += '<offerings campus="UCD" year="2025" term="Autumn">'

with open('AutumnModules.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

current_module_info = []
for line in lines:
    line = line.strip()
    if line == '*******************************':
        numClasses = 0
        lab_or_practical = False
        lecture = False 
        lec_mins = 0
        lab_mins = 0

        if current_module_info:
            for subpart in current_module_info:
                match = pattern.match(subpart)
                altmatch = altpattern.match(subpart)
                credit_match = credit_pattern.match(subpart)
                lecturer_match = lecturer.match(subpart)

                if lecturer_match:
                    coordinator_info = lecturer_match.group(1).strip()
                    parts = coordinator_info.split()

                    if parts[0] == 'Assoc' and len(parts) == 5:
                        firstName = parts[2]
                        lastName = parts[-1]
                    elif parts[0] == 'Dr' and len(parts) == 4:
                        firstName = parts[1]
                        lastName = parts[-1]
                    elif parts[0] == 'Mr' and len(parts) == 4:
                        firstName = parts[1]
                        lastName = parts[-1]
                    elif parts[0] == 'Professor' and len(parts) == 4:
                        firstName = parts[1]
                        lastName = parts[-1]

                    else:
                        firstName = parts[-2]
                        lastName = parts[-1]

                    xml_file = 'AutumnStaffImport.xml'
                    tree = ET.parse(xml_file)
                    root = tree.getroot()
                    for staffMember in root.findall('staffMember'):
                        if (staffMember.get('firstName') == firstName and 
                            staffMember.get('lastName') == lastName):
                            externalId = staffMember.get('externalId')
                            acadTitle = staffMember.get('acadTitle')
                            break
                

                if match:
                    module_code = match.group(1)
                    module_title = match.group(2)
                    subject = module_code[:4]
                    course_nbr = module_code[4:]
                    
                elif altmatch:
                    module_code = altmatch.group(1)
                    module_title = altmatch.group(2)
                    subject = module_code[:3]
                    course_nbr = module_code[3:]

                if credit_match:
                    credit = float(credit_match.group(1))

                if subpart.startswith("Lecture Offering 1") or subpart.startswith("Lecture Offering 3"):
                    startTime = subpart.split()[-3].replace(':', '')
                    endTime = subpart.split()[-1].replace(':', '')
                    lec_mins += int(endTime) - int(startTime)
                    lecture = True

                if subpart.startswith('Tutorial Offering 1') or subpart.startswith('Practical Offering 1') or subpart.startswith('Computer Aided Lab Offering 1') or subpart.startswith('Practical Offering 1') or subpart.startswith('Laboratory Offering 1'):
                    startTime = subpart.split()[-3].replace(':', '')
                    endTime = subpart.split()[-1].replace(':', '')
                    lab_mins += int(endTime) - int(startTime)
                    lab_or_practical = True
                    
            offering = f'{offering_counter:04}' 
            title = f'{module_code} {module_title}'
            
            if module_code not in seen_modules:
                if lecture or lab_or_practical:
                    seen_modules.add(module_code)

                    xml_content += f'\n\t<offering id="{offering}" offered="true" action="update">\n'
                    xml_content += f'\t\t<course id="{offering}" subject="{subject + course_nbr}" courseNbr="{course_nbr}" controlling="true" title="{title}">\n'
                    xml_content += f'\t\t\t<courseCredit creditType="collegiate" creditUnitType="semesterHours" creditFormat="fixedUnit" fixedCredit="{credit}"/>\n'
                    xml_content += '\t\t</course>\n'
                    xml_content += '\t\t<config name="1" limit="120">\n'
                        
                    if lecture:
                        xml_content += f'\t\t\t<subpart type="Lec" suffix="" minPerWeek="50"/>\n'
                        xml_content += f'\t\t\t<class id="{class_counter}" type="Lec" suffix="1" limit="120">\n'
                        xml_content += f'\t\t\t\t<instructor id="{externalId}" fname="{firstName}" lname="{lastName}" title="{acadTitle}" share="100" lead="true"/>\n'
                        xml_content += f'\t\t\t</class>\n'
                        if lec_mins > 50:
                            xml_content += f'\t\t\t<class id="{class_counter}" type="Lec" suffix="2" limit="120">\n'
                            xml_content += f'\t\t\t\t<instructor id="{externalId}" fname="{firstName}" lname="{lastName}" title="{acadTitle}" share="100" lead="true"/>\n'
                            xml_content += f'\t\t\t</class>\n'
                        if not lab_or_practical:
                            class_counter += 1

                    if(lab_or_practical):
                        xml_content += f'\t\t\t<subpart type="Lab" suffix="" minPerWeek="50"/>\n'
                        xml_content += f'\t\t\t<class id="{class_counter}" type="Lab" suffix="1" limit="120">\n'
                        xml_content += f'\t\t\t\t<instructor id="{externalId}" fname="{firstName}" lname="{lastName}" title="{acadTitle}" share="100" lead="true"/>\n'
                        xml_content += f'\t\t\t</class>\n'
                        if lab_mins > 50:
                            xml_content += f'\t\t\t<class id="{class_counter}" type="Lab" suffix="2" limit="120">\n'
                            xml_content += f'\t\t\t\t<instructor id="{externalId}" fname="{firstName}" lname="{lastName}" title="{acadTitle}" share="100" lead="true"/>\n'
                            xml_content += f'\t\t\t</class>\n'
                        class_counter += 1

                    xml_content += '\t\t</config>\n'
                    xml_content += '\t</offering>\n'
                
                offering_counter += 1
                lecture = False
                lab_or_practical = False

            current_module_info = []
        continue
    else:
        current_module_info.append(line)

xml_content += '</offerings>\n'

with open('AutumnOffering.xml', 'w', encoding='utf-8') as output_file:
    output_file.write(xml_content)

print('XML file "courseOffering.xml" has been generated successfully.')


XML file "courseOffering.xml" has been generated successfully.


Now for Spring

In [9]:
pattern = re.compile(r'^([A-Z]{4}\d+)\s(.+)$')
altpattern = re.compile(r'^([A-Z]{3}\d+)\s(.+)$')
credit_pattern = re.compile(r'^Credits:\s+(\d+\.\d+)$')
lecture_pattern = re.compile(r'^Lectures\s+(\d+)$')
lecturer = re.compile(r'^Module Coordinator:\s*(.*)$')
seen_modules = set()
offering_counter = 1
class_counter = 1000

xml_content = '<?xml version="1.0" encoding="UTF-8"?>\n'
xml_content += '<!DOCTYPE offerings PUBLIC "-//UniTime//DTD University Course Timetabling/EN" "http://www.unitime.org/interface/CourseOfferingExport.dtd">\n\n'
xml_content += '<!-- \n'
xml_content += ' * Licensed to The Apereo Foundation under one or more contributor license\n'
xml_content += ' * agreements. See the NOTICE file distributed with this work for\n'
xml_content += ' * additional information regarding copyright ownership.\n'
xml_content += ' *\n'
xml_content += ' * The Apereo Foundation licenses this file to you under the Apache License,\n'
xml_content += ' * Version 2.0 (the "License"); you may not use this file except in\n'
xml_content += ' * compliance with the License. You may obtain a copy of the License at:\n'
xml_content += ' *\n'
xml_content += ' * http://www.apache.org/licenses/LICENSE-2.0\n'
xml_content += ' *\n'
xml_content += ' * Unless required by applicable law or agreed to in writing, software\n'
xml_content += ' * distributed under the License is distributed on an "AS IS" BASIS,\n'
xml_content += ' * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n'
xml_content += ' *\n'
xml_content += ' * See the License for the specific language governing permissions and\n'
xml_content += ' * limitations under the License.\n'
xml_content += ' *\n'
xml_content += ' -->\n\n'
xml_content += '<offerings campus="UCD" year="2025" term="Spring">'

with open('SpringModules.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

current_module_info = []
for line in lines:
    line = line.strip()
    if line == '*******************************':
        numClasses = 0
        lab_or_practical = False
        lecture = False 
        lec_mins = 0
        lab_mins = 0

        if current_module_info:
            for subpart in current_module_info:
                match = pattern.match(subpart)
                altmatch = altpattern.match(subpart)
                credit_match = credit_pattern.match(subpart)
                lecturer_match = lecturer.match(subpart)

                if lecturer_match:
                    coordinator_info = lecturer_match.group(1).strip()
                    parts = coordinator_info.split()

                    if parts[0] == 'Assoc' and len(parts) == 5:
                        firstName = parts[2]
                        lastName = parts[-1]
                    elif parts[0] == 'Dr' and len(parts) == 4:
                        firstName = parts[1]
                        lastName = parts[-1]
                    elif parts[0] == 'Mr' and len(parts) == 4:
                        firstName = parts[1]
                        lastName = parts[-1]
                    elif parts[0] == 'Professor' and len(parts) == 4:
                        firstName = parts[1]
                        lastName = parts[-1]

                    else:
                        firstName = parts[-2]
                        lastName = parts[-1]

                    xml_file = 'SpringStaffImport.xml'
                    tree = ET.parse(xml_file)
                    root = tree.getroot()
                    for staffMember in root.findall('staffMember'):
                        if (staffMember.get('firstName') == firstName and 
                            staffMember.get('lastName') == lastName):
                            externalId = staffMember.get('externalId')
                            acadTitle = staffMember.get('acadTitle')
                            break
                

                if match:
                    module_code = match.group(1)
                    module_title = match.group(2)
                    subject = module_code[:4]
                    course_nbr = module_code[4:]
                    
                elif altmatch:
                    module_code = altmatch.group(1)
                    module_title = altmatch.group(2)
                    subject = module_code[:3]
                    course_nbr = module_code[3:]

                if credit_match:
                    credit = float(credit_match.group(1))

                if subpart.startswith("Lecture Offering 1") or subpart.startswith("Lecture Offering 3"):
                    startTime = subpart.split()[-3].replace(':', '')
                    endTime = subpart.split()[-1].replace(':', '')
                    lec_mins += int(endTime) - int(startTime)
                    lecture = True

                if subpart.startswith('Tutorial Offering 1') or subpart.startswith('Practical Offering 1') or subpart.startswith('Computer Aided Lab Offering 1') or subpart.startswith('Practical Offering 1') or subpart.startswith('Laboratory Offering 1'):
                    startTime = subpart.split()[-3].replace(':', '')
                    endTime = subpart.split()[-1].replace(':', '')
                    lab_mins += int(endTime) - int(startTime)
                    lab_or_practical = True
                    
            offering = f'{offering_counter:04}' 
            title = f'{module_code} {module_title}'
            
            if module_code not in seen_modules:
                if lecture or lab_or_practical:
                    seen_modules.add(module_code)

                    xml_content += f'\n\t<offering id="{offering}" offered="true" action="update">\n'
                    xml_content += f'\t\t<course id="{offering}" subject="{subject + course_nbr}" courseNbr="{course_nbr}" controlling="true" title="{title}">\n'
                    xml_content += f'\t\t\t<courseCredit creditType="collegiate" creditUnitType="semesterHours" creditFormat="fixedUnit" fixedCredit="{credit}"/>\n'
                    xml_content += '\t\t</course>\n'
                    xml_content += '\t\t<config name="1" limit="120">\n'
                        
                    if lecture:
                        xml_content += f'\t\t\t<subpart type="Lec" suffix="" minPerWeek="50"/>\n'
                        xml_content += f'\t\t\t<class id="{class_counter}" type="Lec" suffix="1" limit="120">\n'
                        xml_content += f'\t\t\t\t<instructor id="{externalId}" fname="{firstName}" lname="{lastName}" title="{acadTitle}" share="100" lead="true"/>\n'
                        xml_content += f'\t\t\t</class>\n'
                        if lec_mins > 50:
                            xml_content += f'\t\t\t<class id="{class_counter}" type="Lec" suffix="2" limit="120">\n'
                            xml_content += f'\t\t\t\t<instructor id="{externalId}" fname="{firstName}" lname="{lastName}" title="{acadTitle}" share="100" lead="true"/>\n'
                            xml_content += f'\t\t\t</class>\n'
                        if not lab_or_practical:
                            class_counter += 1

                    if(lab_or_practical):
                        xml_content += f'\t\t\t<subpart type="Lab" suffix="" minPerWeek="50"/>\n'
                        xml_content += f'\t\t\t<class id="{class_counter}" type="Lab" suffix="1" limit="120">\n'
                        xml_content += f'\t\t\t\t<instructor id="{externalId}" fname="{firstName}" lname="{lastName}" title="{acadTitle}" share="100" lead="true"/>\n'
                        xml_content += f'\t\t\t</class>\n'
                        if lab_mins > 50:
                            xml_content += f'\t\t\t<class id="{class_counter}" type="Lab" suffix="2" limit="120">\n'
                            xml_content += f'\t\t\t\t<instructor id="{externalId}" fname="{firstName}" lname="{lastName}" title="{acadTitle}" share="100" lead="true"/>\n'
                            xml_content += f'\t\t\t</class>\n'
                        class_counter += 1

                    xml_content += '\t\t</config>\n'
                    xml_content += '\t</offering>\n'
                
                offering_counter += 1
                lecture = False
                lab_or_practical = False

            current_module_info = []
        continue
    else:
        current_module_info.append(line)

xml_content += '</offerings>\n'

with open('SpringOffering.xml', 'w', encoding='utf-8') as output_file:
    output_file.write(xml_content)

print('XML file "courseOffering.xml" has been generated successfully.')


XML file "courseOffering.xml" has been generated successfully.


Now we must create the preferences for autumn and spring, starting with aututmn. This assigns time patterns to each class, allowing it to be assigned through the solver within the Unitime Software

In [10]:
def generate_class_xml(class_info):
    class_xml = f'  <class subject="{class_info["subject"]}" course="{class_info["course"]}" type="{class_info["type"]}" suffix="{class_info["suffix"]}" externalId="{class_info["id"]}">\n'
    class_xml += f'  \t<timePref pattern="1 x 50" level="R">\n'
    class_xml += f'  \t\t<pref days="{class_info["day"]}" time="{class_info["startTime"]}" level="-1"/>\n'
    class_xml += f'  \t</timePref>\n'
    class_xml += '  </class>\n\n'
    if class_info["duration"] > 50:
            class_xml += f'  <class subject="{class_info["subject"]}" course="{class_info["course"]}" type="{class_info["type"]}" suffix="{class_info["suffix"]}" externalId="{class_info["id"]}">\n'
            class_xml += f'  \t<timePref pattern="1 x 50" level="R">\n'
            secondOffering = int(endTime) - 50
            class_xml += f'  \t\t<pref days="{class_info["day"]}" time="{secondOffering}" level="-1"/>\n'
            class_xml += f'  \t</timePref>\n'
            class_xml += '  </class>\n\n'
    return class_xml

def generate_exact_class(class_info):
    class_xml = f'  <class subject="{class_info["subject"]}" course="{class_info["course"]}" type="{class_info["type"]}" suffix="{class_info["suffix"]}" externalId="{class_info["id"]}">\n'
    class_xml += f'  \t<timePref pattern="Exact Time" level="R">\n'
    class_xml += f'  \t\t<pref days="{class_info["day"]}" time="{class_info["startTime"]}" level="R"/>\n'
    class_xml += f'  \t</timePref>\n'
    class_xml += '  </class>\n\n'
    if class_info["duration"] > 50:
            class_xml += f'  <class subject="{class_info["subject"]}" course="{class_info["course"]}" type="{class_info["type"]}" suffix="{class_info["suffix"]}" externalId="{class_info["id"]}">\n'
            class_xml += f'  \t<timePref pattern="Exact Time" level="R">\n'
            secondOffering = int(endTime) - 50
            class_xml += f'  \t\t<pref days="{class_info["day"]}" time="{secondOffering}" level="R"/>\n'
            class_xml += f'  \t</timePref>\n'
            class_xml += '  </class>\n\n'
    return class_xml

capture_lines = False
edge_case = False
extracted_lines = []
added_classes = set()
added_comp_classes = set()
tree = ET.parse('AutumnOffering.xml')
root = tree.getroot()

xml_content = '<?xml version="1.0" encoding="UTF-8"?>\n'
xml_content += '<!DOCTYPE preferences PUBLIC "-//UniTime//UniTime Preferences/EN" "http://www.unitime.org/interface/Preferences.dtd">\n\n'
xml_content += '<!-- \n'
xml_content += ' * Licensed to The Apereo Foundation under one or more contributor license\n'
xml_content += ' * agreements. See the NOTICE file distributed with this work for\n'
xml_content += ' * additional information regarding copyright ownership.\n'
xml_content += ' *\n'
xml_content += ' * The Apereo Foundation licenses this file to you under the Apache License,\n'
xml_content += ' * Version 2.0 (the "License"); you may not use this file except in\n'
xml_content += ' * compliance with the License. You may obtain a copy of the License at:\n'
xml_content += ' *\n'
xml_content += ' * http://www.apache.org/licenses/LICENSE-2.0\n'
xml_content += ' *\n'
xml_content += ' * Unless required by applicable law or agreed to in writing, software\n'
xml_content += ' * distributed under the License is distributed on an "AS IS" BASIS,\n'
xml_content += ' * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n'
xml_content += ' *\n'
xml_content += ' * See the License for the specific language governing permissions and\n'
xml_content += ' * limitations under the License.\n'
xml_content += ' *\n'
xml_content += ' -->\n\n'

xml_content += '<preferences term="Autumn" year="2025" campus="UCD" dateFormat="yyyy/M/d" timeFormat="HHmm" created="Wed Mar 08 08:56:15 CET 2017">\n'
xml_content += '  <department code="0001">\n'
xml_content += '  </department>\n'

for offering in root.findall('offering'):
    for course in offering.findall('course'):
        subject = course.get('subject')
        course = course.get('courseNbr')
    if subject[:4] != 'COMP':
        extracted_lines = []
        with open('AutumnModules.txt', 'r', encoding='utf-8') as file:
            lines = file.readlines()

            for line in lines:
                line = line.strip()

                if line.startswith(subject):
                    capture_lines = True 

                if capture_lines:                
                    if line.startswith('Lecture Offering 1') or line.startswith('Tutorial Offering 1') or line.startswith('Practical Offering 1') or line.startswith('Computer Aided Lab Offering 1') or line.startswith('Practical Offering 1') or line.startswith('Laboratory Offering 1'):
                        extracted_lines.append(line)

                if line == '*******************************':
                    capture_lines = False  
            if extracted_lines != []:
                for each_line in extracted_lines:
                    if each_line.startswith("Lecture"):
                        type = 'Lec'
                        info = each_line.split()[-4] + ' ' + each_line.split()[-3] + ' ' + each_line.split()[-2] + ' ' + each_line.split()[-1]
                        fullday = info.split()[0]
                        if fullday.split()[0].startswith('Th'):
                            day = 'R'
                        else:
                            day = fullday.split()[0][0]
                        startTime = info.split()[1].replace(':', '')
                        endTime = info.split()[3].replace(':', '')
                        duration = int(endTime) - int(startTime)

                        for config in offering.findall('config'):
                            class_elems = config.findall('class')
                            
                            for i, class_elem in enumerate(class_elems):
                                if class_elem.get('type') == 'Lec':
                                    suffix = class_elem.get('suffix')
                                    id = class_elem.get('id')

                                    class_info = {
                                        'subject': subject,
                                        'course': course,
                                        'type': type,
                                        'suffix': suffix,
                                        'id': id,
                                        'startTime': startTime,
                                        'endTime': endTime,
                                        'duration': duration,
                                        'day': day
                                    }

                                    class_identifier = (
                                        class_info['subject'],
                                        class_info['suffix'],
                                        class_info['id'],
                                        class_info['type'],
                                        class_info['day'],
                                        class_info['startTime'],
                                        class_info['endTime']
                                    )

                                    if class_identifier not in added_classes:
                                        xml_content += generate_exact_class(class_info)
                                    added_classes.add(class_identifier)

                    else:
                        type = 'Lab'
                        info = each_line.split()[-4] + ' ' + each_line.split()[-3] + ' ' + each_line.split()[-2] + ' ' + each_line.split()[-1]
                        fullday = info.split()[0]
                        if fullday.split()[0].startswith('Th'):
                            day = 'R'
                        else:
                            day = fullday.split()[0][0]
                        startTime = info.split()[1].replace(':', '')
                        endTime = info.split()[3].replace(':', '')
                        duration = int(endTime) - int(startTime)

                        for config in offering.findall('config'):
                            class_elems = config.findall('class')
                            
                            for i, class_elem in enumerate(class_elems):
                                if class_elem.get('type') == 'Lab':
                                    suffix = class_elem.get('suffix')
                                    id = class_elem.get('id')

                                    class_info = {
                                        'subject': subject,
                                        'course': course,
                                        'type': type,
                                        'suffix': suffix,
                                        'id': id,
                                        'startTime': startTime,
                                        'endTime': endTime,
                                        'duration': duration,
                                        'day': day
                                    }

                                    class_identifier = (
                                        class_info['subject'],
                                        class_info['suffix'],
                                        class_info['id'],
                                        class_info['type'],
                                        class_info['day'],
                                        class_info['startTime'],
                                        class_info['endTime']
                                    )

                                    if class_identifier not in added_classes:
                                        xml_content += generate_exact_class(class_info)
                                        added_classes.add(class_identifier)


    else:
        extracted_lines = []
        with open('AutumnModules.txt', 'r', encoding='utf-8') as file:
            lines = file.readlines()

            for line in lines:
                line = line.strip()

                if line.startswith(subject):
                    capture_lines = True 

                if capture_lines:                
                    if line.startswith('Lecture Offering 1') or line.startswith('Lecture Offering 3') or line.startswith('Tutorial Offering 1') or line.startswith('Practical Offering 1') or line.startswith('Computer Aided Lab Offering 1') or line.startswith('Practical Offering 1') or line.startswith('Laboratory Offering 1'):
                        extracted_lines.append(line)

                if line == '*******************************':
                    capture_lines = False
            if extracted_lines != []:
                for each_line in extracted_lines:
                    if each_line.startswith("COMP20070"):
                        edge_case = True
                    if each_line.startswith("Lecture"):
                        type = 'Lec'
                        info = each_line.split()[-4] + ' ' + each_line.split()[-3] + ' ' + each_line.split()[-2] + ' ' + each_line.split()[-1]
                        fullday = info.split()[0]
                        if fullday.split()[0].startswith('Th'):
                            day = 'R'
                        else:
                            day = fullday.split()[0][0]
                        startTime = info.split()[1].replace(':', '')
                        endTime = info.split()[3].replace(':', '')
                        duration = int(endTime) - int(startTime)

                        for config in offering.findall('config'):
                            class_elems = config.findall('class')
                            
                            for i, class_elem in enumerate(class_elems):
                                if class_elem.get('type') == 'Lec':
                                    suffix = class_elem.get('suffix')
                                    id = class_elem.get('id')

                                    class_info = {
                                        'subject': subject,
                                        'course': course,
                                        'type': type,
                                        'suffix': suffix,
                                        'id': id,
                                        'startTime': startTime,
                                        'endTime': endTime,
                                        'duration': duration,
                                        'day': day
                                    }

                                    class_identifier = (
                                        class_info['subject'],
                                        class_info['suffix'],
                                        class_info['id'],
                                        class_info['type'],
                                        class_info['day'],
                                        class_info['startTime'],
                                        class_info['endTime']
                                    )

                                    if class_identifier not in added_classes:
                                        xml_content += generate_class_xml(class_info)
                                    added_classes.add(class_identifier)

                    else:
                        type = 'Lab'
                        info = each_line.split()[-4] + ' ' + each_line.split()[-3] + ' ' + each_line.split()[-2] + ' ' + each_line.split()[-1]
                        fullday = info.split()[0]
                        if fullday.split()[0].startswith('Th'):
                            day = 'R'
                        else:
                            day = fullday.split()[0][0]
                        startTime = info.split()[1].replace(':', '')
                        endTime = info.split()[3].replace(':', '')
                        duration = int(endTime) - int(startTime)

                        for config in offering.findall('config'):
                            class_elems = config.findall('class')
                            
                            for i, class_elem in enumerate(class_elems):
                                if class_elem.get('type') == 'Lab':
                                    suffix = class_elem.get('suffix')
                                    id = class_elem.get('id')

                                    class_info = {
                                        'subject': subject,
                                        'course': course,
                                        'type': type,
                                        'suffix': suffix,
                                        'id': id,
                                        'startTime': startTime,
                                        'endTime': endTime,
                                        'duration': duration,
                                        'day': day
                                    }

                                    class_identifier = (
                                        class_info['subject'],
                                        class_info['suffix'],
                                        class_info['id'],
                                        class_info['type'],
                                        class_info['day'],
                                        class_info['startTime'],
                                        class_info['endTime']
                                    )

                                    if class_identifier not in added_classes:
                                        xml_content += generate_class_xml(class_info)
                                        added_classes.add(class_identifier)

xml_content += '</preferences>\n'

with open('AutumnPreferences.xml', 'w') as output_file:
    output_file.write(xml_content)

print('XML file "AutumnPreferences.xml" has been generated successfully.')


XML file "AutumnPreferences.xml" has been generated successfully.


Now Spring

In [11]:
def generate_class_xml(class_info):
    class_xml = f'  <class subject="{class_info["subject"]}" course="{class_info["course"]}" type="{class_info["type"]}" suffix="{class_info["suffix"]}" externalId="{class_info["id"]}">\n'
    class_xml += f'  \t<timePref pattern="1 x 50" level="R">\n'
    class_xml += f'  \t\t<pref days="{class_info["day"]}" time="{class_info["startTime"]}" level="-1"/>\n'
    class_xml += f'  \t</timePref>\n'
    class_xml += f'  </class>\n'
    if class_info["duration"] > 50:
            class_xml += f'  <class subject="{class_info["subject"]}" course="{class_info["course"]}" type="{class_info["type"]}" suffix="{class_info["suffix"]}" externalId="{class_info["id"]}">\n'
            class_xml += f'  \t<timePref pattern="1 x 50" level="R">\n'
            secondOffering = int(endTime) - 50
            class_xml += f'  \t\t<pref days="{class_info["day"]}" time="{secondOffering}" level="-1"/>\n'
            class_xml += f'  \t</timePref>\n'
            class_xml += '  </class>\n\n'
    return class_xml

def generate_exact_class(class_info):
    class_xml = f'  <class subject="{class_info["subject"]}" course="{class_info["course"]}" type="{class_info["type"]}" suffix="{class_info["suffix"]}" externalId="{class_info["id"]}">\n'
    class_xml += f'  \t<timePref pattern="Exact Time" level="R">\n'
    class_xml += f'  \t\t<pref days="{class_info["day"]}" time="{class_info["startTime"]}" level="R"/>\n'
    class_xml += f'  \t</timePref>\n'
    class_xml += '  </class>\n\n'
    if class_info["duration"] > 50:
            class_xml += f'  <class subject="{class_info["subject"]}" course="{class_info["course"]}" type="{class_info["type"]}" suffix="{class_info["suffix"]}" externalId="{class_info["id"]}">\n'
            class_xml += f'  \t<timePref pattern="Exact Time" level="R">\n'
            secondOffering = int(endTime) - 50
            class_xml += f'  \t\t<pref days="{class_info["day"]}" time="{secondOffering}" level="R"/>\n'
            class_xml += f'  \t</timePref>\n'
            class_xml += '  </class>\n\n'
    return class_xml

capture_lines = False
extracted_lines = []
added_classes = set()
added_comp_classes = set()

tree = ET.parse('SpringOffering.xml')
root = tree.getroot()

xml_content = '<?xml version="1.0" encoding="UTF-8"?>\n'
xml_content += '<!DOCTYPE preferences PUBLIC "-//UniTime//UniTime Preferences/EN" "http://www.unitime.org/interface/Preferences.dtd">\n\n'
xml_content += '<!-- \n'
xml_content += ' * Licensed to The Apereo Foundation under one or more contributor license\n'
xml_content += ' * agreements. See the NOTICE file distributed with this work for\n'
xml_content += ' * additional information regarding copyright ownership.\n'
xml_content += ' *\n'
xml_content += ' * The Apereo Foundation licenses this file to you under the Apache License,\n'
xml_content += ' * Version 2.0 (the "License"); you may not use this file except in\n'
xml_content += ' * compliance with the License. You may obtain a copy of the License at:\n'
xml_content += ' *\n'
xml_content += ' * http://www.apache.org/licenses/LICENSE-2.0\n'
xml_content += ' *\n'
xml_content += ' * Unless required by applicable law or agreed to in writing, software\n'
xml_content += ' * distributed under the License is distributed on an "AS IS" BASIS,\n'
xml_content += ' * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n'
xml_content += ' *\n'
xml_content += ' * See the License for the specific language governing permissions and\n'
xml_content += ' * limitations under the License.\n'
xml_content += ' *\n'
xml_content += ' -->\n\n'
xml_content += '<preferences term="Spring" year="2025" campus="UCD" dateFormat="yyyy/M/d" timeFormat="HHmm" created="Wed Mar 08 08:56:15 CET 2017">\n'
xml_content += '  <department code="0001">\n'
xml_content += '  </department>\n'

for offering in root.findall('offering'):
    for course in offering.findall('course'):
        subject = course.get('subject')
        course = course.get('courseNbr')
    if subject[:4] != 'COMP':
        extracted_lines = []
        with open('SpringModules.txt', 'r', encoding='utf-8') as file:
            lines = file.readlines()

            for line in lines:
                line = line.strip()

                if line.startswith(subject):
                    capture_lines = True  

                if capture_lines:                
                    if line.startswith('Lecture Offering 1') or line.startswith('Tutorial Offering 1') or line.startswith('Practical Offering 1') or line.startswith('Computer Aided Lab Offering 1') or line.startswith('Practical Offering 1') or line.startswith('Laboratory Offering 1'):
                        extracted_lines.append(line)

                if line == '*******************************':
                    capture_lines = False 

            if extracted_lines != []:
                for each_line in extracted_lines:
                    if each_line.startswith("Lecture"):
                        type = 'Lec'
                        info = each_line.split()[-4] + ' ' + each_line.split()[-3] + ' ' + each_line.split()[-2] + ' ' + each_line.split()[-1]
                        fullday = info.split()[0]
                        if fullday.split()[0].startswith('Th'):
                            day = 'R'
                        else:
                            day = fullday.split()[0][0]
                        startTime = info.split()[1].replace(':', '')
                        endTime = info.split()[3].replace(':', '')
                        duration = int(endTime) - int(startTime)

                        for config in offering.findall('config'):
                            class_elems = config.findall('class')
                            
                            for i, class_elem in enumerate(class_elems):
                                if class_elem.get('type') == 'Lec':
                                    suffix = class_elem.get('suffix')
                                    id = class_elem.get('id')

                                    class_info = {
                                        'subject': subject,
                                        'course': course,
                                        'type': type,
                                        'suffix': suffix,
                                        'id': id,
                                        'startTime': startTime,
                                        'endTime': endTime,
                                        'duration': duration,
                                        'day': day
                                    }

                                    class_identifier = (
                                        class_info['subject'],
                                        class_info['suffix'],
                                        class_info['id'],
                                        class_info['type'],
                                        class_info['day'],
                                        class_info['startTime'],
                                        class_info['endTime']
                                    )

                                    if class_identifier not in added_classes:
                                        xml_content += generate_exact_class(class_info)
                                    added_classes.add(class_identifier)

                    else:
                        type = 'Lab'
                        info = each_line.split()[-4] + ' ' + each_line.split()[-3] + ' ' + each_line.split()[-2] + ' ' + each_line.split()[-1]
                        fullday = info.split()[0]
                        if fullday.split()[0].startswith('Th'):
                            day = 'R'
                        else:
                            day = fullday.split()[0][0]
                        startTime = info.split()[1].replace(':', '')
                        endTime = info.split()[3].replace(':', '')
                        duration = int(endTime) - int(startTime)

                        for config in offering.findall('config'):
                            class_elems = config.findall('class')
                            
                            for i, class_elem in enumerate(class_elems):
                                if class_elem.get('type') == 'Lab':
                                    suffix = class_elem.get('suffix')
                                    id = class_elem.get('id')

                                    class_info = {
                                        'subject': subject,
                                        'course': course,
                                        'type': type,
                                        'suffix': suffix,
                                        'id': id,
                                        'startTime': startTime,
                                        'endTime': endTime,
                                        'duration': duration,
                                        'day': day
                                    }

                                    class_identifier = (
                                        class_info['subject'],
                                        class_info['suffix'],
                                        class_info['id'],
                                        class_info['type'],
                                        class_info['day'],
                                        class_info['startTime'],
                                        class_info['endTime']
                                    )

                                    if class_identifier not in added_classes:
                                        xml_content += generate_exact_class(class_info)
                                        added_classes.add(class_identifier)


    else:
        extracted_lines = []
        with open('SpringModules.txt', 'r', encoding='utf-8') as file:
            lines = file.readlines()

            for line in lines:
                line = line.strip()

                if line.startswith(subject):
                    capture_lines = True  
                    
                if capture_lines:                
                    if line.startswith('Lecture Offering 1') or line.startswith('Tutorial Offering 1') or line.startswith('Practical Offering 1') or line.startswith('Computer Aided Lab Offering 1') or line.startswith('Practical Offering 1') or line.startswith('Laboratory Offering 1'):
                        extracted_lines.append(line)

                if line == '*******************************':
                    capture_lines = False  
            if extracted_lines != []:
                for each_line in extracted_lines:
                    if each_line.startswith("Lecture"):
                        type = 'Lec'
                        info = each_line.split()[-4] + ' ' + each_line.split()[-3] + ' ' + each_line.split()[-2] + ' ' + each_line.split()[-1]
                        fullday = info.split()[0]
                        if fullday.split()[0].startswith('Th'):
                            day = 'R'
                        else:
                            day = fullday.split()[0][0]
                        startTime = info.split()[1].replace(':', '')
                        endTime = info.split()[3].replace(':', '')
                        duration = int(endTime) - int(startTime)

                        for config in offering.findall('config'):
                            class_elems = config.findall('class')
                            
                            for i, class_elem in enumerate(class_elems):
                                if class_elem.get('type') == 'Lec':
                                    suffix = class_elem.get('suffix')
                                    id = class_elem.get('id')

                                    class_info = {
                                        'subject': subject,
                                        'course': course,
                                        'type': type,
                                        'suffix': suffix,
                                        'id': id,
                                        'startTime': startTime,
                                        'endTime': endTime,
                                        'duration': duration,
                                        'day': day
                                    }

                                    class_identifier = (
                                        class_info['subject'],
                                        class_info['suffix'],
                                        class_info['id'],
                                        class_info['type'],
                                        class_info['day'],
                                        class_info['startTime'],
                                        class_info['endTime']
                                    )

                                    if class_identifier not in added_classes:
                                        xml_content += generate_class_xml(class_info)
                                    added_classes.add(class_identifier)

                    else:
                        type = 'Lab'
                        info = each_line.split()[-4] + ' ' + each_line.split()[-3] + ' ' + each_line.split()[-2] + ' ' + each_line.split()[-1]
                        fullday = info.split()[0]
                        if fullday.split()[0].startswith('Th'):
                            day = 'R'
                        else:
                            day = fullday.split()[0][0]
                        startTime = info.split()[1].replace(':', '')
                        endTime = info.split()[3].replace(':', '')
                        duration = int(endTime) - int(startTime)

                        for config in offering.findall('config'):
                            class_elems = config.findall('class')
                            
                            for i, class_elem in enumerate(class_elems):
                                if class_elem.get('type') == 'Lab':
                                    suffix = class_elem.get('suffix')
                                    id = class_elem.get('id')

                                    class_info = {
                                        'subject': subject,
                                        'course': course,
                                        'type': type,
                                        'suffix': suffix,
                                        'id': id,
                                        'startTime': startTime,
                                        'endTime': endTime,
                                        'duration': duration,
                                        'day': day
                                    }

                                    class_identifier = (
                                        class_info['subject'],
                                        class_info['suffix'],
                                        class_info['id'],
                                        class_info['type'],
                                        class_info['day'],
                                        class_info['startTime'],
                                        class_info['endTime']
                                    )

                                    if class_identifier not in added_classes:
                                        xml_content += generate_class_xml(class_info)
                                        added_classes.add(class_identifier)

xml_content += '</preferences>\n'

with open('SpringPreferences.xml', 'w') as output_file:
    output_file.write(xml_content)

print('XML file "SpringPreferences.xml" has been generated successfully.')


XML file "SpringPreferences.xml" has been generated successfully.


We want to separate cores and option modules, between each year through distribution preferences - cores are prohibited to be at the same time, options are discouraged to be at the same time.

In [12]:
stage = 1
stage_checker = 1
course_counter = 0
option = 0
core = False
current_stage_info = []
current_course_info = []
pattern = re.compile(r'^([A-Z]{4}\d+)\s(.+)$')
altpattern = re.compile(r'^([A-Z]{3}\d+)\s(.+)$')

with open('AutumnPreferences.xml', 'r') as old_autumn_file, open('AutumnPreferencesNew.xml', 'w') as new_autumn_file:
    for line in old_autumn_file:
        new_autumn_file.write(line)
        
        if '<department ' in line:
            with open('AutumnModules.txt', 'r') as ModuleInformation:
                for module_line in ModuleInformation:
                    if ' ******************************* ' in module_line:
                        for course_line in current_course_info:
                            match = pattern.match(course_line)
                            alt_match = altpattern.match(course_line)
                            if match or alt_match:
                                module_code = course_line.split( )[0]
                                course_nbr = course_line[4:9]
                            if course_line.startswith('Lecture Offering 1'):
                                startTime = course_line.split()[-3].replace(':', '')
                                endTime = course_line.split()[-1].replace(':', '')
                                if int(endTime) - int(startTime) == 150:
                                    new_autumn_file.write('    <distributionPref type="BTB" structure="AllClasses" level="-1">\n')
                                    new_autumn_file.write(f'      <subpart subject="{module_code}" course="{course_nbr}" type="Lec"/>\n')
                                    new_autumn_file.write('    </distributionPref>\n')
                            elif course_line.startswith('Tutorial Offering 1') or course_line.startswith('Practical Offering 1') or course_line.startswith('Computer Aided Lab Offering 1') or course_line.startswith('Practical Offering 1') or course_line.startswith('Laboratory Offering 1'):
                                startTime = course_line.split()[-3].replace(':', '')
                                endTime = course_line.split()[-1].replace(':', '')
                                if int(endTime) - int(startTime) == 150:
                                    new_autumn_file.write('    <distributionPref type="BTB" structure="AllClasses" level="-1">\n')
                                    new_autumn_file.write(f'      <subpart subject="{module_code}" course="{course_nbr}" type="Lab"/>\n')
                                    new_autumn_file.write('    </distributionPref>\n')
                        current_course_info = []
                    else:
                        current_course_info.append(module_line)
                                
            new_autumn_file.write('    <distributionPref type="SAME_D_T" structure="AllClasses" level="P">\n')
            
            with open('AutumnModules.txt', 'r') as ModuleInformation:
                for module_line in ModuleInformation:
                    if module_line.startswith('Stage'):
                        stage = module_line.split( )[1]
                        if int(stage) == stage_checker and int(stage) != 1:
                            new_autumn_file.write('    </distributionPref>\n')
                            if(core):
                                new_autumn_file.write('    <distributionPref type="SAME_D_T" structure="AllClasses" level="P">\n')
                            if not core:
                                new_autumn_file.write('    <distributionPref type="SAME_D_T" structure="AllClasses" level="1">\n')
                        
                        if int(stage) < stage_checker and int(stage) != 1:
                            new_autumn_file.write('    </distributionPref>\n')  
                            if(core):
                                new_autumn_file.write('    <distributionPref type="SAME_D_T" structure="AllClasses" level="P">\n')
                            if not core:
                                new_autumn_file.write('    <distributionPref type="SAME_D_T" structure="AllClasses" level="1">\n')
                        
                        for stage_line in current_stage_info:
                            if stage_line.split( ):
                                match = pattern.match(stage_line)
                                alt_match = altpattern.match(stage_line)
                                if match:
                                    module_code = match.group(1)
                                    course_nbr = module_code[4:]
                                    new_autumn_file.write(f'      <subpart subject="{module_code}" course="{course_nbr}" type="Lec"/>\n')
                                    new_autumn_file.write(f'      <subpart subject="{module_code}" course="{course_nbr}" type="Lab"/>\n')
                                if alt_match:
                                    module_code = alt_match.group(1)
                                    course_nbr = module_code[3:]
                                    new_autumn_file.write(f'      <subpart subject="{module_code}" course="{course_nbr}" type="Lec"/>\n')
                                    new_autumn_file.write(f'      <subpart subject="{module_code}" course="{course_nbr}" type="Lab"/>\n')
                        
                        stage_checker = int(stage) + 1

                        if module_line.split( )[2] == 'Core':
                            core = True

                        elif module_line.split( )[2] == 'Options':
                            core = False
                        
                        if 'Stage 1 Options -  A)' in module_line:
                            core = True
                        current_stage_info = []
                    else:
                        current_stage_info.append(module_line)
            
            for stage_line in current_stage_info:
                if stage_line.split( ): 
                    match = pattern.match(stage_line)
                    alt_match = altpattern.match(stage_line)
                    if match:
                        module_code = match.group(1)
                        course_nbr = module_code[4:]
                        new_autumn_file.write(f'      <subpart subject="{module_code}" course="{course_nbr}" type="Lec"/>\n')
                        new_autumn_file.write(f'      <subpart subject="{module_code}" course="{course_nbr}" type="Lab"/>\n')
                    if alt_match:
                        module_code = alt_match.group(1)
                        course_nbr = module_code[3:]
                        new_autumn_file.write(f'      <subpart subject="{module_code}" course="{course_nbr}" type="Lec"/>\n')
                        new_autumn_file.write(f'      <subpart subject="{module_code}" course="{course_nbr}" type="Lab"/>\n')
            new_autumn_file.write('    </distributionPref>\n')

input_file = 'AutumnPreferencesNew.xml'
output_file = 'AutumnPreferencesFinal.xml'

with open(input_file, 'r') as f:
    lines = f.readlines()

modified_lines = []
i = 0
while i < len(lines):
    line = lines[i].strip()

    if line.startswith('<distributionPref') and lines[i+1].strip() == '</distributionPref>':
        i += 2

    else:
        modified_lines.append(lines[i])
        i += 1

with open(output_file, 'w') as f:
    f.write(''.join(modified_lines))
print("Autumn Preferences Final File Created")


Autumn Preferences Final File Created


Now, for Spring

In [13]:
stage = 1
stage_checker = 1
course_counter = 0
option = 0
core = False
current_stage_info = []
pattern = re.compile(r'^([A-Z]{4}\d+)\s(.+)$')
altpattern = re.compile(r'^([A-Z]{3}\d+)\s(.+)$')

with open('SpringPreferences.xml', 'r') as old_spring_file, open('SpringPreferencesNew.xml', 'w') as new_spring_file:
    for line in old_spring_file:
        new_spring_file.write(line)
        
        if '<department ' in line:
            with open('SpringModules.txt', 'r') as ModuleInformation:
                for module_line in ModuleInformation:
                    if ' ******************************* ' in module_line:
                        for course_line in current_course_info:
                            match = pattern.match(course_line)
                            alt_match = altpattern.match(course_line)
                            if match or alt_match:
                                module_code = course_line.split( )[0]
                                course_nbr = course_line[4:9]
                            if course_line.startswith('Lecture Offering 1'):
                                startTime = course_line.split()[-3].replace(':', '')
                                endTime = course_line.split()[-1].replace(':', '')
                                if int(endTime) - int(startTime) == 150:
                                    new_spring_file.write('    <distributionPref type="BTB" structure="AllClasses" level="-1">\n')
                                    new_spring_file.write(f'      <subpart subject="{module_code}" course="{course_nbr}" type="Lec"/>\n')
                                    new_spring_file.write('    </distributionPref>\n')
                            elif course_line.startswith('Tutorial Offering 1') or course_line.startswith('Practical Offering 1') or course_line.startswith('Computer Aided Lab Offering 1') or course_line.startswith('Practical Offering 1') or course_line.startswith('Laboratory Offering 1'):
                                startTime = course_line.split()[-3].replace(':', '')
                                endTime = course_line.split()[-1].replace(':', '')
                                if int(endTime) - int(startTime) == 150:
                                    new_spring_file.write('    <distributionPref type="BTB" structure="AllClasses" level="-1">\n')
                                    new_spring_file.write(f'      <subpart subject="{module_code}" course="{course_nbr}" type="Lab"/>\n')
                                    new_spring_file.write('    </distributionPref>\n')
                        current_course_info = []
                    else:
                        current_course_info.append(module_line)
            
            new_spring_file.write('    <distributionPref type="SAME_D_T" structure="AllClasses" level="P">\n')
            
            with open('SpringModules.txt', 'r') as ModuleInformation:
                for module_line in ModuleInformation:
                    if module_line.startswith('Stage'):
                        stage = module_line.split( )[1]
                        if int(stage) == stage_checker and int(stage) != 1:
                            new_spring_file.write('    </distributionPref>\n')
                            if(core):
                                new_spring_file.write('    <distributionPref type="SAME_D_T" structure="AllClasses" level="P">\n')
                            if not core:
                                new_spring_file.write('    <distributionPref type="SAME_D_T" structure="AllClasses" level="1">\n')
                        
                        if int(stage) < stage_checker and int(stage) != 1:
                            new_spring_file.write('    </distributionPref>\n')  
                            if(core):
                                new_spring_file.write('    <distributionPref type="SAME_D_T" structure="AllClasses" level="P">\n')
                            if not core:
                                new_spring_file.write('    <distributionPref type="SAME_D_T" structure="AllClasses" level="1">\n')
                        
                        for stage_line in current_stage_info:
                            if stage_line.split( ):
                                match = pattern.match(stage_line)
                                alt_match = altpattern.match(stage_line)
                                if match:
                                    module_code = match.group(1)
                                    course_nbr = module_code[4:]
                                    new_spring_file.write(f'      <subpart subject="{module_code}" course="{course_nbr}" type="Lec"/>\n')
                                    new_spring_file.write(f'      <subpart subject="{module_code}" course="{course_nbr}" type="Lab"/>\n')
                                if alt_match:
                                    module_code = alt_match.group(1)
                                    course_nbr = module_code[3:]
                                    new_spring_file.write(f'      <subpart subject="{module_code}" course="{course_nbr}" type="Lec"/>\n')
                                    new_spring_file.write(f'      <subpart subject="{module_code}" course="{course_nbr}" type="Lab"/>\n')
                        
                        stage_checker = int(stage) + 1
                        
                        if module_line.split( )[2] == 'Core':
                            core = True
                        
                        elif module_line.split( )[2] == 'Options':
                            core = False
                        
                        if 'Stage 1 Options -  A)' in module_line:
                            core = True
                        current_stage_info = []
                    else:
                        current_stage_info.append(module_line)
            
            for stage_line in current_stage_info:
                if stage_line.split( ): 
                    match = pattern.match(stage_line)
                    alt_match = altpattern.match(stage_line)
                    if match:
                        module_code = match.group(1)
                        course_nbr = module_code[4:]
                        new_spring_file.write(f'      <subpart subject="{module_code}" course="{course_nbr}" type="Lec"/>\n')
                        new_spring_file.write(f'      <subpart subject="{module_code}" course="{course_nbr}" type="Lab"/>\n')
                    if alt_match:
                        module_code = alt_match.group(1)
                        course_nbr = module_code[3:]
                        new_spring_file.write(f'      <subpart subject="{module_code}" course="{course_nbr}" type="Lec"/>\n')
                        new_spring_file.write(f'      <subpart subject="{module_code}" course="{course_nbr}" type="Lab"/>\n')
            new_spring_file.write('    </distributionPref>\n')

input_file = 'SpringPreferencesNew.xml'
output_file = 'SpringPreferencesFinal.xml'

with open(input_file, 'r') as f:
    lines = f.readlines()

modified_lines = []
i = 0
while i < len(lines):
    line = lines[i].strip()
    
    if line.startswith('<distributionPref') and lines[i+1].strip() == '</distributionPref>':
        i += 2
    
    else:
        modified_lines.append(lines[i])
        i += 1

with open(output_file, 'w') as f:
    f.write(''.join(modified_lines))
print("Spring Preferences Final File Created")


Spring Preferences Final File Created


Lastly, we implement constraints for teacher preferences. This is currently mock data, taken from a txt file in the form: first_name, last_name, day_unavailable, start_time_unavailable, end_time_unavailable.

In [14]:
tree = ET.parse('AutumnStaffImport.xml')
root = tree.getroot()

with open('AutumnPreferencesFinal.xml', 'r', encoding='utf-8') as old_autumn_file, open('AutumnPreferencesNew.xml', 'w', encoding='utf-8') as new_autumn_file:
    for line in old_autumn_file:
        new_autumn_file.write(line)
        
        if '</department>' in line:
            with open('teacherPreferences.txt', 'r', encoding='utf-8') as lecturerInformation:
                for line in lecturerInformation:
                    firstName = line.split()[0]
                    lastName = line.split()[1]
                    day = line.split()[2][0]
                    if line.split()[2][1] == 'h':
                        day = 'R'
                    startTime = line.split()[3].replace(':', '')
                    endTime = line.split()[5].replace(':', '')
                    for staffMember in root.findall('staffMember'):
                        if (staffMember.get('firstName') == firstName and 
                            staffMember.get('lastName') == lastName):
                            externalId = staffMember.get('externalId')
                            acadTitle = staffMember.get('acadTitle')
                            break
                    
                    new_autumn_file.write(f'  <instructor externalId="{externalId}" firstName="{firstName}" lastName="{lastName}" title="{acadTitle}" department="0001">\n')
                    new_autumn_file.write('\t<timePref level="R">\n')
                    new_autumn_file.write(f'\t\t<pref level="P" day="{day}" start="{startTime}" stop="{endTime}"/>\n')
                    new_autumn_file.write('\t</timePref>\n')
                    new_autumn_file.write('  </instructor>\n')

input_file = 'AutumnPreferencesNew.xml'
output_file = 'AutumnPreferencesFinal.xml'

with open(input_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()

modified_lines = []
i = 0
while i < len(lines):
    line = lines[i].strip()
    
    if line.startswith('<distributionPref') and lines[i+1].strip() == '</distributionPref>':
        i += 2
    
    else:
        modified_lines.append(lines[i])
        i += 1

with open(output_file, 'w', encoding='utf-8') as f:
    f.write(''.join(modified_lines))
print("Autumn Preferences Final File Created")


Autumn Preferences Final File Created


In [15]:
tree = ET.parse('SpringStaffImport.xml')
root = tree.getroot()

with open('SpringPreferencesFinal.xml', 'r', encoding='utf-8') as old_autumn_file, open('SpringPreferencesNew.xml', 'w', encoding='utf-8') as new_autumn_file:
    for line in old_autumn_file:
        new_autumn_file.write(line)
        
        if '</department>' in line:
            with open('teacherPreferences.txt', 'r', encoding='utf-8') as lecturerInformation:
                for line in lecturerInformation:
                    firstName = line.split()[0]
                    lastName = line.split()[1]
                    day = line.split()[2][0]
                    if line.split()[2][1] == 'h':
                        day = 'R'
                    startTime = line.split()[3].replace(':', '')
                    endTime = line.split()[5].replace(':', '')
                    for staffMember in root.findall('staffMember'):
                        if (staffMember.get('firstName') == firstName and 
                            staffMember.get('lastName') == lastName):
                            externalId = staffMember.get('externalId')
                            acadTitle = staffMember.get('acadTitle')
                            break
                    
                    new_autumn_file.write(f'  <instructor externalId="{externalId}" firstName="{firstName}" lastName="{lastName}" title="{acadTitle}" department="0001">\n')
                    new_autumn_file.write('\t<timePref level="R">\n')
                    new_autumn_file.write(f'\t\t<pref level="P" day="{day}" start="{startTime}" stop="{endTime}"/>\n')
                    new_autumn_file.write('\t</timePref>\n')
                    new_autumn_file.write('  </instructor>\n')

input_file = 'SpringPreferencesNew.xml'
output_file = 'SpringPreferencesFinal.xml'

with open(input_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()

modified_lines = []
i = 0
while i < len(lines):
    line = lines[i].strip()
    
    if line.startswith('<distributionPref') and lines[i+1].strip() == '</distributionPref>':
        i += 2
    
    else:
        modified_lines.append(lines[i])
        i += 1

with open(output_file, 'w', encoding='utf-8') as f:
    f.write(''.join(modified_lines))
print("Autumn Preferences Final File Created")


Autumn Preferences Final File Created
